# Splitting, Preprocessing and Model Development
This notebook is used for:


### Declaring Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.colors import LogNorm, Normalize
import sklearn
import warnings
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

### Color Palette & Typeface Sizing

In [2]:
YELLOW = '#F2DC5D'
GREEN = '#9BC53D'
DARK_GREEN = '#597222'
RED = '#C3423F'
LIGHT_BLUE = '#2596BE'
GRAY = '#666666'

AXIS_SIZE = 12
TITLE_SIZE = 16
DESCRIPTION_SIZE = 9
FIGURE_SIZE = (10*2/3,6*2/3)

RANDOM_STATE = 14

### Import Dataframe

In [3]:
#****************************************************import dataset****************************************************
df = pd.read_csv('../data/final.csv', dtype={'citizen': 'string', 'sex': 'string', 'age': 'string', 'decision': 'string', 'geo': 'string', 'TIME_PERIOD': 'string', 'GENCONV': "Int64", 'HUMSTAT': "Int64", 'SUB_PROT': "Int64", 'REJECTED': "Int64", 'TOTAL_POS': "Int64", 'TOTAL_APPS': "Int64", "POS_RATE": "Float64"}, keep_default_na=False, na_values=['nan'])

##remove partial 2023-Q3 Data
df = df[df["TIME_PERIOD"] != "2023-Q3"]

df

,citizen,sex,age,geo,TIME_PERIOD,GENCONV,HUMSTAT,SUB_PROT,REJECTED,TOTAL_POS,TOTAL_APPS
0,AD,F,UNK,AT,2008-Q1,0,0,0,0,0,0
1,AD,F,UNK,AT,2008-Q2,0,0,0,0,0,0
2,AD,F,UNK,AT,2008-Q3,0,0,0,0,0,0
3,AD,F,UNK,AT,2008-Q4,0,0,0,0,0,0
4,AD,F,UNK,AT,2009-Q1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
7221109,ZW,UNK,Y_LT14,UK,2019-Q3,0,0,0,0,0,0
7221110,ZW,UNK,Y_LT14,UK,2019-Q4,0,0,0,0,0,0
7221111,ZW,UNK,Y_LT14,UK,2020-Q1,0,0,0,0,0,0
7221112,ZW,UNK,Y_LT14,UK,2020-Q2,0,0,0,0,0,0


## 1. Introduce Lagged Features

In [4]:
#****************************************************re-sort dataframe****************************************************

sort_order = ['citizen', 'sex', 'age', 'geo', 'TIME_PERIOD']
df = df.sort_values(by =sort_order) 

#*********************************************create sequential list of quarters****************************************************

quarters = []
for i in range(2008, 2024):
    quarters.append(str(i) + "-Q1")
    quarters.append(str(i) + "-Q2")
    quarters.append(str(i) + "-Q3")
    quarters.append(str(i) + "-Q4")

#****************************************************lagged features****************************************************

QUARTERS_OF_LAG = (4 * 2)

def add_lagged_features(df, features, maintained_columns, QUARTERS_OF_LAG):
    quarters = np.unique(df["TIME_PERIOD"])
    def lagged_features(target_var, lag_count, unit):
        lagged = pd.DataFrame()
        columns = []
        for i in range(1, lag_count + 1):
            lagged = pd.concat([lagged, target_var.shift(i)], axis=1)
            name = target_var.name
            if (i == 1):
                columns.append(name + " - lag " + str(i) + " " + str(unit))
            else:
                columns.append(name + " - lag " + str(i) + " " + str(unit) + "s")
        lagged.columns = columns
        return lagged.astype('Int64')

    #introduce lag for each feature
    df_lagged = df
    for f in features:
        df_lagged = pd.concat([df_lagged, lagged_features(df[f], QUARTERS_OF_LAG, "quarter")], axis=1)

    #remove all features with less than the lag amount of historical data
    #df_lagged = df_lagged[df_lagged.eq()]
    for i in range(0, QUARTERS_OF_LAG):
        shift_eq = df_lagged.eq(df_lagged.shift())
        keep = shift_eq[maintained_columns[0]]
        for j in range(1, len(maintained_columns)):
            keep = keep & shift_eq[maintained_columns[j]]
        df_lagged = df_lagged[keep]
        #print("lagged i" + str(i) + " of " + str(QUARTERS_OF_LAG))
    
    return df_lagged

df_lagged = add_lagged_features(df, ["TOTAL_POS", "TOTAL_APPS"], ['citizen', 'age', 'sex', 'geo'], QUARTERS_OF_LAG)

df_lagged

,citizen,sex,age,geo,TIME_PERIOD,GENCONV,HUMSTAT,SUB_PROT,REJECTED,TOTAL_POS,...,TOTAL_POS - lag 7 quarters,TOTAL_POS - lag 8 quarters,TOTAL_APPS - lag 1 quarter,TOTAL_APPS - lag 2 quarters,TOTAL_APPS - lag 3 quarters,TOTAL_APPS - lag 4 quarters,TOTAL_APPS - lag 5 quarters,TOTAL_APPS - lag 6 quarters,TOTAL_APPS - lag 7 quarters,TOTAL_APPS - lag 8 quarters
8,AD,F,UNK,AT,2010-Q1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,AD,F,UNK,AT,2010-Q2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,AD,F,UNK,AT,2010-Q3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,AD,F,UNK,AT,2010-Q4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,AD,F,UNK,AT,2011-Q1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7221109,ZW,UNK,Y_LT14,UK,2019-Q3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7221110,ZW,UNK,Y_LT14,UK,2019-Q4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7221111,ZW,UNK,Y_LT14,UK,2020-Q1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7221112,ZW,UNK,Y_LT14,UK,2020-Q2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Lagged Feature Testing

In [5]:
#***********************************************testing equivalence function****************************************************
def equivalent_dfs(df1, df2):
    return (df1.reset_index(drop=True) == df2.reset_index(drop=True)).all().all()

#****************************************************lagging testing****************************************************
print("[*] testing lagging function")
#simple tests
tdf0_data = [['AR', 1, 1], ['AR', 2, 2], ['AR', 3, 3], ['AR', 4, 4], ['AR', 5, 5]]
tdf0 = pd.DataFrame(data=tdf0_data, columns=['citizen', 'TIME_PERIOD', 'val'])
tdf1_data = [['AR', 2, 2, 1], ['AR', 3, 3, 2], ['AR', 4, 4, 3], ['AR', 5, 5, 4]]
tdf1 = pd.DataFrame(data=tdf1_data, columns=['citizen', 'TIME_PERIOD', 'val', 'val - lag 1 quarter'])
tdf2_data = [['AR', 3, 3, 2, 1], ['AR', 4, 4, 3, 2], ['AR', 5, 5, 4, 3]]
tdf2 = pd.DataFrame(data=tdf2_data, columns=['citizen', 'TIME_PERIOD', 'val', 'val - lag 1 quarter', 'val - lag 2 quarters'])
tdf3_data = [['AR', 4, 4, 3, 2, 1], ['AR', 5, 5, 4, 3, 2]]
tdf3 = pd.DataFrame(data=tdf3_data, columns=['citizen', 'TIME_PERIOD', 'val', 'val - lag 1 quarter', 'val - lag 2 quarters', 'val - lag 3 quarters'])

assert equivalent_dfs(add_lagged_features(tdf0, ["val"], ['citizen'], 1), 
                      tdf1), "basic lag test failed, shift=1"
assert equivalent_dfs(add_lagged_features(tdf0, ["val"], ['citizen'], 2), 
                      tdf2), "basic lag test failed, shift=2"
assert equivalent_dfs(add_lagged_features(tdf0, ["val"], ['citizen'], 3), 
                      tdf3), "basic lag test failed, shift=3"
print("\t[+] basic lag tests passed")

#more advanced tests
tdf4_data = [['AU', 1, 1], ['AU', 2, 2], ['AU', 3, 3], ['AU', 4, 4], ['AU', 5, 5],
             ['NZ', 1, 11], ['NZ', 2, 12], ['NZ', 3, 13], ['NZ', 4, 14], ['NZ', 5, 15]]
tdf4 = pd.DataFrame(data=tdf4_data, columns=['citizen', 'TIME_PERIOD', 'val'])
tdf5_data = [['AU', 2, 2, 1], ['AU', 3, 3, 2], ['AU', 4, 4, 3], ['AU', 5, 5, 4],
             ['NZ', 2, 12, 11], ['NZ', 3, 13, 12], ['NZ', 4, 14, 13], ['NZ', 5, 15, 14]]
tdf5 = pd.DataFrame(data=tdf5_data, columns=['citizen', 'TIME_PERIOD', 'val', 'val - lag 1 quarter'])
tdf6_data = [['AU', 3, 3, 2, 1], ['AU', 4, 4, 3, 2], ['AU', 5, 5, 4, 3],
             ['NZ', 3, 13, 12, 11], ['NZ', 4, 14, 13, 12], ['NZ', 5, 15, 14, 13]]
tdf6 = pd.DataFrame(data=tdf6_data, columns=['citizen', 'TIME_PERIOD', 'val', 'val - lag 1 quarter', 'val - lag 2 quarters'])


assert equivalent_dfs(add_lagged_features(tdf4, ["val"], ['citizen'], 1), 
                      tdf5), "lag with shared times test failed, shift=1"
assert equivalent_dfs(add_lagged_features(tdf4, ["val"], ['citizen'], 2), 
                      tdf6), "lag with shared times test failed, shift=2"
print("\t[+] tests with shared times passed")

#very complicated tests
tdf7_data = [['AU', 1, 1], ['AU', 2, 2], ['AU', 3, 3], ['AU', 4, 4], ['AU', 5, 5],
             ['NZ', 2, 12], ['NZ', 3, 13], ['NZ', 4, 14], ['NZ', 5, 15]]
tdf7 = pd.DataFrame(data=tdf7_data, columns=['citizen', 'TIME_PERIOD', 'val'])
tdf8_data = [['AU', 2, 2, 1], ['AU', 3, 3, 2], ['AU', 4, 4, 3], ['AU', 5, 5, 4],
             ['NZ', 3, 13, 12], ['NZ', 4, 14, 13], ['NZ', 5, 15, 14]]
tdf8 = pd.DataFrame(data=tdf8_data, columns=['citizen', 'TIME_PERIOD', 'val', 'val - lag 1 quarter'])
tdf9_data = [['AU', 3, 3, 2, 1], ['AU', 4, 4, 3, 2], ['AU', 5, 5, 4, 3],
             ['NZ', 4, 14, 13, 12], ['NZ', 5, 15, 14, 13]]
tdf9 = pd.DataFrame(data=tdf9_data, columns=['citizen', 'TIME_PERIOD', 'val', 'val - lag 1 quarter', 'val - lag 2 quarters'])
tdf10_data = [['AU', 1, 1], ['AU', 2, 2], ['AU', 3, 3], ['AU', 4, 4], ['AU', 5, 5],
              ['NZ', 2, 12], ['NZ', 3, 13], ['NZ', 4, 14], ['NZ', 5, 15],
              ['FJ', 3, 23], ['FJ', 4, 24], ['FJ', 5, 25], ['FJ', 6, 26], ['FJ', 7, 27], ['FJ', 8, 28],
              ['WS', 4, 34], ['WS', 5, 35]]
tdf10 = pd.DataFrame(data=tdf10_data, columns=['citizen', 'TIME_PERIOD', 'new_val'])
tdf11_data = [['AU', 2, 2, 1], ['AU', 3, 3, 2], ['AU', 4, 4, 3], ['AU', 5, 5, 4],
              ['NZ', 3, 13, 12], ['NZ', 4, 14, 13], ['NZ', 5, 15, 14],
              ['FJ', 4, 24, 23], ['FJ', 5, 25, 24], ['FJ', 6, 26, 25], ['FJ', 7, 27, 26], ['FJ', 8, 28, 27],
              ['WS', 5, 35, 34]]
tdf11 = pd.DataFrame(data=tdf11_data, columns=['citizen', 'TIME_PERIOD', 'new_val', 'new_val - lag 1 quarter'])
tdf12_data = [['AU', 5, 5, 4, 3, 2, 1],
              ['FJ', 7, 27, 26, 25, 24, 23], ['FJ', 8, 28, 27, 26, 25, 24]]
tdf12 = pd.DataFrame(data=tdf12_data, columns=['citizen', 'TIME_PERIOD', 'new_val', 'new_val - lag 1 quarter', 'new_val - lag 2 quarters', 'new_val - lag 3 quarters', 'new_val - lag 4 quarters'])

assert equivalent_dfs(add_lagged_features(tdf7, ["val"], ['citizen'], 1), 
                      tdf8), "lag with 2 different start times test failed, shift=1"
assert equivalent_dfs(add_lagged_features(tdf7, ["val"], ['citizen'], 2), 
                      tdf9), "lag with 2 different start times test failed, shift=2"
assert equivalent_dfs(add_lagged_features(tdf10, ["new_val"], ['citizen'], 1), 
                      tdf11), "lag with many different start times test failed, shift=1"
assert equivalent_dfs(add_lagged_features(tdf10, ["new_val"], ['citizen'], 4), 
                      tdf12), "lag with many different start times test failed, shift=4"
print("\t[+] tests with multiple start and end dates passed")

print("\t[+] \x1b[42mPASSED ALL\x1b[0m lagging tests")

[*] testing lagging function
	[+] basic lag tests passed
	[+] tests with shared times passed
	[+] tests with multiple start and end dates passed
	[+] PASSED ALL lagging tests


## 2. Dataset Splitting

In [13]:
#****************************************************splitting****************************************************
TARGET_VAR = "REJECTED"

y = df_lagged[TARGET_VAR]
X = df_lagged.drop(['GENCONV', 'HUMSTAT', 'SUB_PROT', 'REJECTED', 'TOTAL_POS'], axis=1)
#PLAN:
#of 62 quarters...
#QUARTERS_OF_LAG are lost bc they wont have the needed lagged features
    
new_quarters = [q for q in quarters if q >= quarters[QUARTERS_OF_LAG]]
quarter_count = len(new_quarters) - 1

train_split = 0.6
test_split = 0.2
val_split = 0.2



div_0 = new_quarters[0]
div_1 = new_quarters[int(quarter_count * train_split)]
div_2 = new_quarters[int(quarter_count * (train_split + test_split))]
div_3 = new_quarters[quarter_count]
    
X_train = X[(div_0 <= X["TIME_PERIOD"]) & (X["TIME_PERIOD"] < div_1)]
y_train = y[(div_0 <= X["TIME_PERIOD"]) & (X["TIME_PERIOD"] < div_1)]

X_test = X[(div_1 <= X["TIME_PERIOD"]) & (X["TIME_PERIOD"] < div_2)]
y_test = y[(div_1 <= X["TIME_PERIOD"]) & (X["TIME_PERIOD"] < div_2)]

X_val = X[(div_2 <= X["TIME_PERIOD"]) & (X["TIME_PERIOD"] < div_3)]
y_val = y[(div_2 <= X["TIME_PERIOD"]) & (X["TIME_PERIOD"] < div_3)]

In [7]:
def check_split_sizes(X, train, test, val):
    fails = 0
    print("[*] checking train test val split")
    train_set_qs = set(train["TIME_PERIOD"])
    test_set_qs = set(test["TIME_PERIOD"])
    val_set_qs = set(val["TIME_PERIOD"])
    
    #check for TIME_PERIOD overlap
    shared = (train_set_qs & test_set_qs) | (val_set_qs & test_set_qs) | (train_set_qs & val_set_qs)
    if (len(shared) != 0):
        warnings.warn('\t[-] overlap between train, test, or val time_periods')
        fails+=1
    else:
        print("\t[+] no overlap between train, test, or val TIME_PERIODS")
        
    #check for a fairly even 60/20/20 split
    NAMES = ['train', 'test ', 'val  ']
    TARGETS = [0.6, 0.2, 0.2]
    ALLOWED_FRACTION_ERROR = 0.02
    sizes = [len(train) / len(X), len(test) / len(X), len(val) / len(X)]
    for i in range(0, 3):
        if (np.abs(sizes[i] - TARGETS[i]) < ALLOWED_FRACTION_ERROR):
            print("\t[+] " + str(NAMES[i]) + " is " + str(np.round(sizes[i], 3)) + " of datapoints which is within bounds of its " + str(TARGETS[i]) + " target")
        else:
            warnings.warn("\t[-] " + str(NAMES[i]) + " is " + str(np.round(sizes[i], 3)) + " of datapoints which is out of bounds")
            fails+=1

    if (fails == 0):
        print("\t[*] \x1b[42mPASSED ALL\x1b[0m train test val split tests")
    else:
        print("\t[?] \033[91mFAILED " + str(fails) + "\033[0m train test val split tests")
        

check_split_sizes(X, X_train, X_test, X_val)

[*] checking train test val split
	[+] no overlap between train, test, or val TIME_PERIODS
	[+] train is 0.605 of datapoints which is within bounds of its 0.6 target
	[+] test  is 0.207 of datapoints which is within bounds of its 0.2 target
	[+] val   is 0.187 of datapoints which is within bounds of its 0.2 target
	[*] PASSED ALL train test val split tests


## 3. Feature Scaling

In [8]:
#****************************************************feature scaling********************************************************

ordinal_ftrs = ['age', 'TIME_PERIOD']
ordinal_cats = [['UNK','Y_LT14','Y14-17','Y18-34','Y35-64','Y_GE65'], quarters]
                                                                     #^^i'm using quarters not new_quarters here so that
                                                                     #  the model can still tell where in history this q is
onehot_ftrs = ['citizen', 'geo', 'sex']
minmax_ftrs = []
std_ftrs = []

# collect all the encoders
preprocessor = ColumnTransformer(
    transformers=[
        ('ord', OrdinalEncoder(categories = ordinal_cats), ordinal_ftrs),
        ('onehot', OneHotEncoder(sparse_output=False,handle_unknown='ignore'), onehot_ftrs),
        ('minmax', MinMaxScaler(), minmax_ftrs),
        ('std', StandardScaler(), std_ftrs)])

clf = Pipeline(steps=[('preprocessor', preprocessor)]) # for now we only preprocess 
                                                       # later on we will add other steps here


X_train_prep = clf.fit_transform(X_train)
X_val_prep = clf.transform(X_val)
X_test_prep = clf.transform(X_test)

print(X_train.shape)
print(X_train_prep.shape)
print(X_train_prep)
#X_prep

(3783348, 22)
(3783348, 240)
[[ 0.  8.  1. ...  1.  0.  0.]
 [ 0.  9.  1. ...  1.  0.  0.]
 [ 0. 10.  1. ...  1.  0.  0.]
 ...
 [ 1. 38.  0. ...  0.  0.  1.]
 [ 1. 39.  0. ...  0.  0.  1.]
 [ 1. 40.  0. ...  0.  0.  1.]]


In [ ]:
from sklearn.linear_model import Ridge
ridge_param_grid = {
            'model__alpha': [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100], # no upper bound so the values are evenly spaced in log
            } 

ridge_test_scores, ridge_best_models = MLpipe_RMSE(preprocessor, Ridge, ridge_param_grid)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


In [11]:
#****************************************************training models****************************************************

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.model_selection import ParameterGrid
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

# function for the ML pipeline as outlined above 
def MLpipe_RMSE(preprocessor, ML_algo, param_grid):
    '''
    !This function splits the data to other/test (80/20) and then applies KFold with 4 folds to other.
    !The RMSE is minimized in cross-validation.

    !You should:

    !1. Loop through 10 different random states
    !2. Split your data 
    !3. Fit a model using GridSearchCV with KFold and the predefined Preprocessor 
    !4. Calculate the model's error on the test set 
    !5. Return a list of 10 test scores and 10 best models 
    '''
    
    # lists to be returned 
    test_scores = []
    best_models = []
    
    # your code here...
    for RANDOM_STATE in range(0, 10):
        pipe = Pipeline(steps=[
                ('preprocess', preprocessor),
                ('model', ML_algo())
            ])

        grid = GridSearchCV(pipe, param_grid=param_grid,scoring = 'neg_root_mean_squared_error', \
                            return_train_score = True, n_jobs=-2, verbose=True)

        grid.fit(X_val, y_val)
        
        results = pd.DataFrame(grid.cv_results_)
        
        best_models.append(grid)
        y_pred = grid.predict(X_test)
        test_scores.append(np.sqrt(mean_squared_error(y_pred, y_test)))

    return test_scores, best_models

